<a href="https://colab.research.google.com/github/Schmutzz/PV_Forecast_Platone/blob/master/Kopie_von_analytics_lasse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import sqlite3
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# establish sql connection
db_path = "drive/MyDrive/data/input_data.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()

Import all data from SQL

In [ ]:

#timerange = "Timestamp between '2022-07-19 00:00:00' and '2022-07-20 00:00:00' ORDER by Timestamp"
baseline=[pd.read_sql_query("SELECT * FROM baseline", conn).set_index('Timestamp'),'baseline']
household_batteries=[pd.read_sql_query("SELECT * FROM household_batteries ", conn).set_index('Timestamp'), 'household_batteries']
mb_basic=[pd.read_sql_query("SELECT * FROM mb_basic ", conn).set_index('Timestamp'),'mb_basic']
mb_pvpro_1h = [pd.read_sql_query("SELECT * FROM mb_pvpro_1h ", conn).set_index('Timestamp'), 'mb_pvpro_1h']
mb_clouds =[ pd.read_sql_query("SELECT * FROM mb_clouds ", conn).set_index('Timestamp'), 'mb_clouds']
mb_solar = [pd.read_sql_query("SELECT * FROM mb_solar ", conn).set_index('Timestamp'), 'mb_solar']
mb_sunmoon = [pd.read_sql_query("SELECT * FROM mb_sunmoon ", conn).set_index('Timestamp'), 'mb_sunmoon']
slp = [pd.read_sql_query("SELECT * FROM slp ", conn).set_index('Timestamp'), 'slp']
mb_pvpro_15min = [pd.read_sql_query("SELECT * FROM mb_pvpro_15min ", conn).set_index('Timestamp'), 'mb_pvpro_15min']
wunderground_historical_40 = [pd.read_sql_query("SELECT * FROM wunderground_historical_40 ", conn).set_index('Date'), 'wunderground_historical_40' ]
wunderground_historical_43 = [pd.read_sql_query("SELECT * FROM wunderground_historical_43 ", conn).set_index('Date'), 'wunderground_historical_43' ]


# baseline.name='baseline'
# household_batteries.name ='household_batteries'
# mb_basic.name = 'mb_basic'
# mb_pvpro_1h.name = 'mb_pvpro_1h'
# mb_clouds.name = 'mb_clouds'
# mb_solar.name = 'mb_solar'
# mb_sunmoon.name = 'mb_sunmoon'
# slp.name = 'slp'
# mb_pvpro_15min.name = 'mb_pvpro_15min'
# wunderground_historical_40.name = 'wunderground_historical_40' 
# wunderground_historical_43.name = 'wunderground_historical_43' 
# wunderground_historical_25.name = 'wunderground_historical_25'

df_list=[baseline, household_batteries, mb_basic, mb_pvpro_1h, mb_clouds, mb_solar, mb_sunmoon, slp, mb_pvpro_15min, wunderground_historical_40, wunderground_historical_43]
for df in df_list:
    df[0].index=pd.to_datetime(df[0].index)
    

print all data

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    tempDf = df[0].loc['2022-10-01 00:00:00': '2022-10-02 00:00:00']     #hier Zeitraum auswählen; aber nicht zu groß, sonst schmiert das hier ab
    for c in tempDf.columns:
        if c in ['Baseline in kW' ]:
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=True)
        elif c not in ['rainspot', 'visibility']:  
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=False)

fig.show()
#fig.show(renderer='browser')

Interessante Tage:

    Keine/wenig Verschattung:
        '2022-09-21 00:00:00'
        '2022-10-06 00:00:00'
        
    komplette Verschattung:
        '2022-09-24 00:00:00'
        '2022-04-04 00:00:00'

    teilweise bewölkt:
        '2022-10-17 00:00:00'
        '2022-10-16 00:00:00'
        '2022-10-08 00:00:00'

print selected data

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    tempDf = df[0].loc['2022-09-15 00:00:00': '2022-11-14 00:00:00']     #hier Zeitraum auswählen von bis
    for c in tempDf.columns:
        if c in ['Baseline in kW','solar_radiation', 'pvpower_instant']:             #hier Reihe für 1. Achse auswählen
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=True)
        elif c in [ 'Pascal', 'Heisenberg', 'Einstein', 'Kelvin', 'Tesla']:                                   #hier Reihe für 2. Achse auswählen
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=False)

fig.update_layout(template='plotly')
fig.show(renderer = 'browser')
#pups

PV Vergleich von Max

In [ ]:
df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])
df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', conn)
df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43', conn)
df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, parse_dates=['Timestamp'])

df_mb_15 = df_mb_15.resample('1Min', on='Timestamp').mean().ffill()
result_df = df_mb_15.merge(df_households, on='Timestamp', how='inner')

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=result_df['Timestamp'], y=result_df[['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']]), secondary_y=True)
fig.add_trace(go.Scatter(x=result_df['Timestamp'], y=result_df['pvpower_instant']), secondary_y=False)
fig.show(renderer='browser')

# R2_score
Interpretation:
erklärt Veränderung einer abhängigen Variable

In [ ]:
# read in all PV-related data -> interpolation: 1min timesteps / mean: 15min timesteps
resample_method = 'mean'

df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])
df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', conn, parse_dates=['Date'])
df_station40.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43', conn, parse_dates=['Date'])
df_station43.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, parse_dates=['Timestamp'])

if resample_method == 'interpolation':
    df_mb_15 = df_mb_15.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_households.merge(df_mb_15, on='Timestamp', how='inner')
elif resample_method == 'mean':
    df_households = df_households.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_mb_15.merge(df_households, on='Timestamp', how='inner')
else:
    print('WRONG RESAMPLE METHOD!!!')
    raise KeyboardInterrupt

df_results = df_results.merge(df_station40, on='Timestamp', how='inner')
df_results = df_results.merge(df_station43, on='Timestamp', how='inner', suffixes=('_40', '_43'))
df_results['sum_households'] = df_results[['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']].sum(axis=1)
df_results['sum_meteo_error_normalized']=(df_results['pvpower_instant'] / df_results['pvpower_instant'].max()) - (df_results['sum_households'] / df_results['sum_households'].max())
df_results['sum_meteo_error']=df_results['pvpower_instant']-df_results['sum_households']

del df_households, df_station40, df_station43, df_mb_15

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

cols = ['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla', 'sum_households']
for col in cols:
    fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results[col]/df_results[col].max(), name=col+'_normalized', opacity=0.7))
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['pvpower_instant'] / df_results['pvpower_instant'].max(), name='MeteoBlue',
                         opacity=0.7))
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['solar_radiation_40'] / df_results['solar_radiation_40'].max(),
                         name='Wunderground_40', opacity=0.7))
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['solar_radiation_43'] / df_results['solar_radiation_43'].max(),
                         name='Wunderground_43', opacity=0.7))
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['sum_meteo_error_normalized'],
                         name='sum_meteo_err_normalized', opacity=0.7)) 
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['sum_meteo_error'],
                         name='sum_meteo_err', opacity=0.7), secondary_y=True) 
fig.update_layout(title_text=f'Comparison PV values (resample: {resample_method})', title_x=0.5, template='plotly')
fig.update_yaxes(title_text='Household Batteries', secondary_y=False)
fig.update_yaxes(title_text='MeteoBlue & Wunderground (normalized)', secondary_y=True)
fig.show(renderer='notebook')

In [ ]:
# R2 score
def r2_per_group(data, truth, predicted):
    return r2_score(data[truth], data[predicted])


r2_base = 'pvpower_instant'
fig = make_subplots(specs=[[{'secondary_y': True}]])


cols = ['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla', 'sum_households']
df_r2 = pd.DataFrame(columns=cols)
df_results.dropna(inplace=True)

for col in cols:
    df_r2[col] = df_results.groupby(df_results['Timestamp'].dt.date).apply(r2_per_group, col, r2_base)

#print(df_r2)
fig.add_traces([go.Scatter(x=df_r2.index, y=df_r2[col], name=col, opacity=0.7, mode='lines+markers') for col in cols], secondary_ys=[False] * len(cols))
fig.update_yaxes(title_text='Correlation value', range=[-1, 1], secondary_y=False)

"""
cols_clouds = ['lowclouds', 'midclouds', 'highclouds', 'totalcloudcover']
fig.add_traces([go.Scatter(x=df_corr.index, y=df_corr[col], name=col, opacity=0.7, mode='lines+markers') for col in cols_clouds],
               secondary_ys=[True] * len(cols_clouds))
fig.update_yaxes(title_text='Cloud cover in %', range=[0, 100], secondary_y=True)

fig.update_layout(title_text='Daily Correlation to MeteoBlue Forecast vs. Cloud Cover (mean)', title_x=0.5,
                  xaxis_title='Date', template='plotly')
"""

fig.update_layout(title_text=f'Daily Correlation to MeteoBlue Forecast (resample: {resample_method})', title_x=0.5,
                  xaxis_title='Date', template='plotly')

fig.show(renderer='notebook')

# Correlation

In [ ]:
# read in mb clouds and merge into resampled data

df_mb_clouds = pd.read_sql_query('SELECT * FROM mb_clouds', conn, parse_dates=['Timestamp'])

if resample_method == 'interpolation':
    df_mb_clouds = df_mb_clouds.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
elif resample_method == 'mean':
    df_mb_clouds = df_mb_clouds.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
else:
    print('WRONG RESAMPLE METHOD!!!')
    raise KeyboardInterrupt

df_results = df_results.merge(df_mb_clouds, on='Timestamp', how='inner')

del df_mb_clouds

In [ ]:
# correlate every days measured power

corr_target = 'pvpower_instant'
fig = make_subplots(specs=[[{'secondary_y': True}]])

df_corr = pd.DataFrame()
cols = ['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla', 'sum_households']
for col in cols:
    df_corr = pd.concat([df_corr, df_results.groupby(df_results['Timestamp'].dt.date)[col].corr(df_results[corr_target])], axis=1)

df_corr = pd.concat([df_corr, df_results.groupby(df_results['Timestamp'].dt.date)[['lowclouds', 'midclouds', 'highclouds', 'totalcloudcover']].mean()], axis=1)

fig.add_traces([go.Scatter(x=df_corr.index, y=df_corr[col], name=col, opacity=0.7, mode='lines+markers') for col in cols], secondary_ys=[False] * len(cols))
fig.update_yaxes(title_text='Correlation value', range=[0, 1], secondary_y=False)

"""
cols_clouds = ['lowclouds', 'midclouds', 'highclouds', 'totalcloudcover']
fig.add_traces([go.Scatter(x=df_corr.index, y=df_corr[col], name=col, opacity=0.7, mode='lines+markers') for col in cols_clouds],
               secondary_ys=[True] * len(cols_clouds))
fig.update_yaxes(title_text='Cloud cover in %', range=[0, 100], secondary_y=True)

fig.update_layout(title_text='Daily Correlation to MeteoBlue Forecast vs. Cloud Cover (mean)', title_x=0.5,
                  xaxis_title='Date', template='plotly')
"""

fig.update_layout(title_text=f'Daily Correlation to MeteoBlue Forecast (resample: {resample_method})', title_x=0.5,
                  xaxis_title='Date', template='plotly')

fig.show(renderer='notebook')

# Kalman Filter


In [ ]:
# read in all PV-related data -> interpolation: 1min timesteps / mean: 15min timesteps
resample_method = 'interpolation'

df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])
df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', conn, parse_dates=['Date'])
df_station40.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43', conn, parse_dates=['Date'])
df_station43.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, parse_dates=['Timestamp'])

if resample_method == 'interpolation':
    df_mb_15 = df_mb_15.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_households.merge(df_mb_15, on='Timestamp', how='inner')
elif resample_method == 'mean':
    df_households = df_households.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_mb_15.merge(df_households, on='Timestamp', how='inner')
else:
    print('WRONG RESAMPLE METHOD!!!')
    raise KeyboardInterrupt

df_results = df_results.merge(df_station40, on='Timestamp', how='inner')
df_results = df_results.merge(df_station43, on='Timestamp', how='inner', suffixes=('_40', '_43'))
df_results['sum_households'] = df_results[['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']].sum(axis=1)
df_results['sum_meteo_error_normalized']=(df_results['pvpower_instant'] / df_results['pvpower_instant'].max()) - (df_results['sum_households'] / df_results['sum_households'].max())
df_results['sum_meteo_error']=df_results['pvpower_instant']-df_results['sum_households']

del df_households, df_station40, df_station43, df_mb_15

In [ ]:
from filterpy.kalman import KalmanFilter

f = KalmanFilter (dim_x=2, dim_z=1)
#set initial values
f.x = np.array([0.,    # pv power
                0.])   # change of pv power
#state transition matrix
f.F = np.array([[1.,1.],
                [0.,1.]])
#state transition function
f.H = np.array([[1.,0.]])
#covariance matrix 1000=uncertainty    also possible:    f.P *= 1000.
P=1000.
f.P = np.array([[P,    0.],
                [   0., P] ])
#measurement noise
f.R = 5


df=df_results[['pvpower_instant', 'sum_households', 'Timestamp', 'solar_radiation_43']]

df['sum_normalized']=df_results['sum_households']/df_results['sum_households'].max()
df['meteo_normalized']=df_results['pvpower_instant'] / df_results['pvpower_instant'].max()
df['error']=df['meteo_normalized']-df['sum_normalized']
i=0
for idx, row in df.iterrows():
    
    z = row['sum_normalized']
    # print(z)
    df.loc[idx,'kalman']=f.x[0]
    f.predict()
    f.update(z)
    
    #print(f.x)
    i+=1
    print(f.x[0])
    # if i==5:
    #     break
print(df[['kalman', 'pvpower_instant', 'sum_households']])
df['kalmanError']=df['kalman']-df['sum_normalized']
df.set_index('Timestamp', inplace=True)

In [ ]:

fig = make_subplots(specs=[[{'secondary_y': True}]])

cols = ['sum_normalized', 'meteo_normalized', 'error', 'kalman', 'kalmanError']
for col in cols:
    fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col, opacity=0.7))

fig.add_trace(go.Scatter(x=df.index, y=df['sum_normalized'],
                         name='sum_meteo_err', opacity=0.7), secondary_y=False) 
fig.add_trace(go.Scatter(x=df.index, y=df['solar_radiation_43']/df['solar_radiation_43'].max(),
                         name='solar_radiation_43', opacity=0.7), secondary_y=False)                     

fig.show(renderer='notebook')


# Vergleich Wonderground43 und household_batteries

import data

In [ ]:
# read in all PV-related data -> interpolation: 1min timesteps / mean: 15min timesteps
resample_interval = '15Min'

df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', 
                                  conn, parse_dates=['Timestamp'], index_col='Timestamp')
df_households = df_households.resample(resample_interval).mean().ffill(limit=1).interpolate()

df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43_long_real', 
                                 conn, parse_dates=['Date'], index_col='Date')
df_station43.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_station43 = df_station43.resample(resample_interval).mean().ffill(limit=1).interpolate()

df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', 
                                 conn, parse_dates=['Date'], index_col='Date')
df_station40.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_station40 = df_station40.resample(resample_interval).mean().ffill(limit=1).interpolate()

df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', 
                             conn, parse_dates=['Timestamp'], index_col='Timestamp')
df_mb_15 = df_mb_15.resample(resample_interval).mean().ffill(limit=1).interpolate()

df_mb_clouds = pd.read_sql_query('SELECT Timestamp, totalcloudcover FROM mb_clouds', 
                                 conn, parse_dates=['Timestamp'], index_col='Timestamp')
df_mb_clouds = df_mb_clouds.resample(resample_interval).mean().ffill(limit=1).interpolate()

df_results=pd.merge(left=df_households, right=df_station40, how='inner', left_index=True, right_index=True)
df_results=pd.merge(left=df_results, right=df_station43, how='inner', suffixes=('_40', '_43'), left_index=True, right_index=True)
df_results=pd.merge(left=df_results, right=df_mb_15, how='inner', left_index=True, right_index=True)
df_results=pd.merge(left=df_results, right=df_mb_clouds, how='inner', left_index=True, right_index=True)
df_results['Date'] = df_results.index

# df_results=addTimeDelta(df_results)
# df_results=addMissingValsColumn(df_results)

# df_results = df_results.resample(resample_interval).mean().ffill(limit=1).interpolate()
# df_results.dropna(inplace=True)
# df_results['measuredBool']=df_results.loc(df_results['missingValsBelow']==0).shift(period=1)

df_results['sum_households'] = df_results[['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']].sum(axis=1)
df_results=pd.concat([df_results.loc['2022-09-16 00:00:00':'2022-10-18 00:00:00'],df_results.loc['2022-10-26 00:00:00':'2022-11-15 00:00:00']])

wrong_dates = [pd.Timestamp(day=22, month=9, year=2022), pd.Timestamp(day=4, month=10, year=2022), pd.Timestamp(day=5, month=10, year=2022)]
df_results = df_results.loc[~df_results['Date'].dt.date.isin(wrong_dates)].reset_index()
df_results = df_results.loc[df_results['Date'].dt.date != pd.Timestamp(day=4, month=10, year=2022)]

del df_households, df_station40

/usr/local/lib/python3.8/dist-packages/pandas/core/algorithms.py:524: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/usr/local/lib/python3.8/dist-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



In [ ]:

fig = make_subplots(specs=[[{'secondary_y': True}]])

cols = ['solar_radiation', 'sum_households']

fig.add_trace(go.Scatter(x=df_results.index, y=df_results['solar_radiation_40'], name='solar_radiation_40', opacity=0.7), secondary_y=False)
fig.add_trace(go.Scatter(x=df_results.index, y=df_results['solar_radiation_43'], name='solar_radiation_43', opacity=0.7), secondary_y=False)
fig.add_trace(go.Scatter(x=df_results.index, y=df_results['sum_households'], name='sum_households', opacity=0.7), secondary_y=True)
fig.add_trace(go.Scatter(x=df_results.index, y=df_results['pvpower_instant'], name='pvpower_instant', opacity=0.7), secondary_y=False)

fig.show()

In [ ]:
# df=df_results[df_results.index.hour > 11]
df=df_results
fig = px.scatter(df, x='solar_radiation_43', y='sum_households', opacity=0.7, hover_name='Date', template='plotly', trendline='ols')
fig.update_layout(title='Wunderground 43 vs. Households', title_x=0.5, xaxis_title='Solar radiation', yaxis_title='PV-Power')
fig.show()

In [ ]:
df_results['regression_43'] = (df_results['solar_radiation_43'] * 45.77) + 375.5

In [ ]:
"""
fig = go.Figure()
split_hour = 9

before = df_results.loc[df_results['Date'].dt.hour <= split_hour]
fig.add_trace(go.Scatter(x=before['solar_radiation_40'], 
                         y=before['sum_households'], name=f'<= {split_hour}', 
                         opacity=0.7, mode='markers'))

after = df_results.loc[df_results['Date'].dt.hour > 11]
fig.add_trace(go.Scatter(x=after['solar_radiation_40'], 
                         y=after['sum_households'], name=f'> {split_hour}', 
                         opacity=0.7, mode='markers'))
fig.update_layout(title='Wunderground 40 vs. Households', title_x=0.5, xaxis_title='Solar radiation', yaxis_title='PV-Power')

"""
# df=df_results[df_results.index.hour >= 11]
df=df_results
fig = px.scatter(after, x='solar_radiation_40', y='sum_households', opacity=0.7, hover_name='Date', template='plotly', trendline='ols')
fig.update_layout(title='Wunderground 40 vs. Households (> 9am)', title_x=0.5, xaxis_title='Solar radiation', yaxis_title='PV-Power')

fig.show()

In [ ]:
df_results['regression_40'] = (df_results['solar_radiation_40'] * 45.66) + 713.8

In [ ]:
before['regression_40'] = (before['solar_radiation_40'] * 91.37) + 380.3
after['regression_40'] = (after['solar_radiation_40'] * 43.82) + 202.7

In [ ]:
after['delta_40'] = after['regression_40'] - after['sum_households']

<ipython-input-60-9cffabaee01c>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_results.to_csv('drive/MyDrive/data/regression_results.csv', index=False, 
                  columns=['Date', 'sum_households', 'solar_radiation_43', 
                           'solar_radiation_40', 'regression_43', 'regression_40', 
                           'delta_43', 'delta_40'], decimal=',', sep=';')

before.to_csv('drive/MyDrive/data/regression_results_until_9am.csv', index=False, 
              columns=['Date', 'sum_households', 'solar_radiation_40', 'regression_40', 'delta_40'], 
              decimal=',', sep=';')

after.to_csv('drive/MyDrive/data/regression_results_after_9am.csv', index=False, 
              columns=['Date', 'sum_households', 'solar_radiation_40', 'regression_40', 'delta_40'], 
              decimal=',', sep=';')

In [ ]:
fig = px.scatter(x=df_results['pvpower_instant'], y=df_results['sum_households'], opacity=0.6, template='plotly', trendline='ols')
fig.update_layout(title='MeteoBlue vs. Households', title_x=0.5, xaxis_title='PV-Forecast', yaxis_title='PV-Power')

fig.show()

In [ ]:
filtered = df_results.loc[df_results.index.hour > 13]
filtered['sum_households'].corr(filtered['solar_radiation_40'])

0.9081402203143113

In [ ]:
from sklearn.metrics import mean_squared_error
results = pd.DataFrame()
for i in range(24):
    filtered = df_results.loc[df_results['Date'].dt.hour == i]
    
    wg40 = filtered['sum_households'].corr(filtered['solar_radiation_40'])    
    wg43 = filtered['sum_households'].corr(filtered['solar_radiation_43'])    
    mb = filtered['sum_households'].corr(filtered['pvpower_instant'])
    """
    wg40 = mean_squared_error(filtered['sum_households'], filtered['solar_radiation_40'])
    wg43 = mean_squared_error(filtered['sum_households'], filtered['solar_radiation_43'])
    mb = mean_squared_error(filtered['sum_households'], filtered['pvpower_instant'])
    """
    results = pd.concat([results, pd.DataFrame({'Wunderground_40': [wg40], 'Wunderground_43': [wg43], 'MeteoBlue': [mb]}, index=[i])])

In [ ]:
fig = px.line(results[5:17], y=results.columns, log_y=False)
fig.update_layout(title='Correlation to Households depending on time of day', title_x=0.5,
                  xaxis_title='Hour of Day', yaxis_title='Correlation')
fig.show()

In [ ]:
test = df_results.loc[df_results.index.hour >= 17]

In [ ]:
test['sum_households'].corr(test['pvpower_instant'])

0.6598977737251809

In [ ]:
test.isna().sum()

In [ ]:
test['pvpower_instant'].value_counts()

In [ ]:
# alle zeitpunkte
df_results[['sum_households', 'pvpower_instant', 'solar_radiation_43', 'solar_radiation_40']].corr()

,sum_households,pvpower_instant,solar_radiation_43,solar_radiation_40
sum_households,1.000000,0.929314,0.906573,0.850692
pvpower_instant,0.929314,1.000000,0.868691,0.805908
solar_radiation_43,0.906573,0.868691,1.000000,0.896797
solar_radiation_40,0.850692,0.805908,0.896797,1.000000


In [ ]:
# zeitpunkte bis 9 uhr
before[['sum_households', 'pvpower_instant', 'solar_radiation_40']].corr()

,sum_households,pvpower_instant,solar_radiation_40
sum_households,1.000000,0.943443,0.798613
pvpower_instant,0.943443,1.000000,0.844259
solar_radiation_40,0.798613,0.844259,1.000000


In [ ]:
# zeitpunkte ab 9 uhr
after[['sum_households', 'pvpower_instant', 'solar_radiation_40']].corr()

,sum_households,pvpower_instant,solar_radiation_40
sum_households,1.000000,0.906762,0.947331
pvpower_instant,0.906762,1.000000,0.878143
solar_radiation_40,0.947331,0.878143,1.000000
